In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Filter WFS on property

The wfs downloaded is the muncipality of Amsterdam, Netherlands. The wfs is provided by [pdok](https://www.pdok.nl/introductie/-/article/bestuurlijke-grenzen).

In [2]:
from pathlib import Path

import geopandas as gpd
from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService

from building_detector import helpers


wfs_url = 'https://geodata.nationaalgeoregister.nl/bestuurlijkegrenzen/wfs'

In [3]:
wfs = WebFeatureService(wfs_url, version='1.1.0')
wfs.identification.title

'Bestuurlijke grenzen WFS'

In [4]:
print('The WFS has the following layers')
list(wfs.contents)

The WFS has the following layers


['bestuurlijkegrenzen:gemeenten',
 'bestuurlijkegrenzen:landsgrens',
 'bestuurlijkegrenzen:provincies']

We need the `'bestuurlijkegrenzen:gemeenten'` ('gemeenten' is 'municipalities' in Dutch).

In [5]:
layer_name = 'bestuurlijkegrenzen:gemeenten'
wfs.get_schema(layer_name)

{'properties': {'code': 'string', 'gemeentenaam': 'string'},
 'geometry': 'GeometryCollection',
 'geometry_column': 'geom'}

We will use the `'gemeentenaam'` - municipality name - property to filter for `'Amsterdam'`.

In [6]:
muni = 'Amsterdam'
filter_ = PropertyIsEqualTo(
    propertyname='gemeentenaam', 
    literal=muni, 
    matchcase=True,
)
filterxml = etree.tostring(filter_.toXML()).decode("utf-8")

response = wfs.getfeature(typename=layer_name, filter=filterxml)

muni_path = Path(f'../data/{muni}.gml'.lower())
muni_path.write_bytes(bytes(response.read(), 'UTF-8'))

121886

Get the bounding box - envelope -  of the layer.

In [7]:
envelope = gpd.read_file(muni_path).envelope
helpers.Bbox.from_poly(envelope[0])

BBox(xmin=110188.384, ymin=476770.797, xmax=134029.81, ymax=493893.636)

In [8]:
envelope_path = muni_path.parent / f'{muni_path.stem}_envelope.gml'
envelope.to_file(envelope_path, driver='GML')